# Youtube Video Summarization

## My First Frontier LLM Project!

Welcome to my first LLM-based project! The goal of this project is to leverage large language models (LLMs) to summarize YouTube videos. Currently, it only supports English transcriptions, so instead of watching the entire video, you can simply read the summary!

## Important Note
Be mindful when testing with longer videos, as they may consume significant resources and could lead to high costs on your ChatGPT bill.
You can switch to Ollama for free usage if you're looking to reduce costs.


In [ ]:
!pip install youtube-transcript-api openai

In [111]:
# imports

import os

import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display

from openai import OpenAI
from youtube_transcript_api import YouTubeTranscriptApi
import re

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [ ]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


In [113]:
openai = OpenAI()

In [114]:
class YoutubeVideoID:
    def __init__(self, url):
        self.url = url
        self.video_id = self.extract_video_id(url)

    def extract_video_id(self, url):
        """
        Extracts the YouTube video ID from a given URL.
        Supports both regular and shortened URLs.
        """
        # Regular expression to match YouTube video URL and extract the video ID
        regex = r"(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|\S*\?v=)|(?:youtu\.be\/))([a-zA-Z0-9_-]{11})"
        match = re.match(regex, url)
        
        if match:
            return match.group(1)
        else:
            raise ValueError("Invalid YouTube URL")

    def __str__(self):
        return f"Video ID: {self.video_id}"

In [ ]:
# Example usage
video_url = "https://www.youtube.com/watch?v=5zuF4Ys1eAw"

yt_video = YoutubeVideoID(video_url)
print(yt_video)  # Output: Video ID: cicHKo4zH-w

In [116]:
def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return " ".join([item['text'] for item in transcript])
    except Exception as e:
        print(f"Error fetching transcript: {e}")
        return None

In [ ]:
# Fetch transcript using the video ID
transcript_text = get_transcript(yt_video.video_id)
print(len(transcript_text))

In [118]:
# Function to summarize text using ChatGPT
def summarize_text(text):
    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes text."},
                {"role": "user", "content": f"Summarize the following text:\n{text}"}
            ],
            max_tokens=200
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error summarizing text: {e}")
        return None

In [ ]:
def split_text(text, chunk_size=3000):
    """Splits large text into smaller chunks based on the given chunk size."""
    chunks = []
    while len(text) > chunk_size:
        # Find the last full stop within the chunk size to avoid cutting sentences
        split_point = text.rfind('.', 0, chunk_size)
        
        # If no full stop is found, just split at the chunk size
        if split_point == -1:
            split_point = chunk_size
        
        chunks.append(text[:split_point].strip())
        text = text[split_point:].strip()
    
    # Add the remaining text as the final chunk
    if text:
        chunks.append(text)
    
    return chunks

transcript_chunks = split_text(transcript_text)
# Now you can summarize each chunk individually
summaries = []
for chunk in transcript_chunks:
    summary = summarize_text(chunk)
    summaries.append(summary)
    
# Combine the individual summaries into one
full_summary = " ".join(summaries)
display(Markdown(full_summary))
